# **Extracting and Visualizing Stock Data Description**

## **Table of Contents**



*   Define Function
*   Using yfinance to Extract Tesla Stock Data
*   Using Webscraping to Extract Tesla Revenue Data
*   Using yfinance to Extract GameStop Stock Data
*   Using Webscraping to Extract GME Revenue Data
*   Plotting Stock Graph

## **Installs and Imports**

In [1]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
import plotly.io as pio
pio.renderers.default = "iframe"

In [4]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## **Define Function**

Defined the function `make_graph`. It takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.

In [5]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

## **Using yfinance to Extract Tesla Stock Data**

Using the `Ticker` function to enter the ticker symbol of the stock we want to extract data on. The stock is Tesla and its ticker symbol is `TSLA`.

In [6]:
tesla = yf.Ticker("TSLA")

Using the ticker object and the function `history` to extract stock information and save it in a dataframe named `tesla_data`.

In [7]:
tesla_data = tesla.history(period="max")

Reseting the index using the `reset_index(inplace=True)` function on the tesla_data DataFrame and displaying the first five rows of the `tesla_data` dataframe using the `head` function.

In [8]:
tesla_data.reset_index(inplace=True)

In [9]:
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## **Using Webscraping to Extract Tesla Revenue Data**

Using the `requests` library to download the webpage, and parsing the html data using `beautiful_soup` using parser i.e `html5lib` or `html.parser`.

In [10]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text

In [11]:
soup = BeautifulSoup(html_data, "html.parser")

Using `BeautifulSoup` or the `read_html` function to extract the table with `Tesla Revenue` and store it into a dataframe named `tesla_revenue`. The dataframe have columns `Date` and `Revenue`.

In [12]:
rows = []
tbody = soup.find_all("tbody")[1]

for tr in tbody.find_all("tr"):
    cols = tr.find_all("td")
    if len(cols) >= 2:
        date = cols[0].get_text(strip=True)
        revenue_text = cols[1].get_text(strip=True)

        rows.append({"Date": date, "Revenue": revenue_text})

tesla_revenue = pd.DataFrame(rows, columns=['Date', 'Revenue'])

Removing the comma and dollar sign from the `Revenue` column.

In [13]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"",regex=True)

<>:1: SyntaxWarning:

invalid escape sequence '\$'

<>:1: SyntaxWarning:

invalid escape sequence '\$'

/tmp/ipython-input-3152827612.py:1: SyntaxWarning:

invalid escape sequence '\$'



Removing null or empty strings in the Revenue column.

In [14]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [15]:
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


## **Using yfinance to Extract GameStop Stock Data**

Using the `Ticker` function enter the ticker symbol of the chosen stock to extract data on to was created a ticker object. The stock is GameStop and its ticker symbol is `GME`.

In [16]:
gme = yf.Ticker("GME")

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `gme_data`.

In [17]:
gme_data = gme.history(period="max")

In [18]:
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


## **Using Webscraping to Extract GME Revenue Data**

Using the `requests` library to download the webpage https//cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html

In [19]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data_2 = requests.get(url).text

Parsing the html data using `beautiful_soup` using parser i.e `html5lib` or `html.parser`.

In [20]:
soup2 = BeautifulSoup(html_data_2, "html.parser")

Using `BeautifulSoup` or the `read_html` function to extract the table with `GameStop Revenue` and store it into a dataframe named `gme_revenue`. Removing the comma and dollar sign from the `Revenue` column.

In [21]:
rows = []
tbody = soup2.find_all("tbody")[1]

for tr in tbody.find_all("tr"):
    cols = tr.find_all("td")
    if len(cols) >= 2:
        date = cols[0].get_text(strip=True)
        revenue_text = cols[1].get_text(strip=True)

        rows.append({"Date": date, "Revenue": revenue_text})

gme_revenue = pd.DataFrame(rows, columns=['Date', 'Revenue'])

gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$', "", regex=True)

<>:14: SyntaxWarning:

invalid escape sequence '\$'

<>:14: SyntaxWarning:

invalid escape sequence '\$'

/tmp/ipython-input-673571864.py:14: SyntaxWarning:

invalid escape sequence '\$'



In [22]:
gme_revenue.tail()

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


## **Plotting Stock Graph**

Using the `make_graph` function to graph the Tesla Stock Data.

In [23]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

Output hidden; open in https://colab.research.google.com to view.

Using the `make_graph` function to graph the GameStop Stock Data.

In [25]:
make_graph(gme_data, gme_revenue, 'GameStop')

Output hidden; open in https://colab.research.google.com to view.